In [ ]:
import numpy as np
import pandas as pd
import pickle

In [ ]:
SUBMISSION_TYPE = 'validation'

In [ ]:
top_forecast = pd.read_csv('../data/refined/forecast_prophet_store_dept.csv')
top_forecast = top_forecast[top_forecast.y.isnull()]
top_forecast['store_id'] = top_forecast['id'].str[:4]
top_forecast['dept_id'] = top_forecast['id'].str[5:]
top_forecast.head()

In [ ]:
#top_down_ratio = pd.read_csv('../data/refined/top_down_ratio_' + SUBMISSION_TYPE + '.csv')
top_down_ratio = pd.read_csv('../data/external/lgb_weights_validation.csv')
top_down_ratio

In [ ]:
#top_down_ratio.describe()[['sales_ratio']]

In [ ]:
#top_down_ratio.groupby(['dept_id', 'store_id', 'd'])['sales_ratio'].sum().reset_index()

In [ ]:
#from scipy.special import softmax
#
#top_down_ratio['sales_ratio_softmax'] = test \
#    .groupby(['dept_id', 'store_id', 'd'])['sales_ratio'] \
#    .apply(lambda x: softmax(x))
#
#top_down_ratio['sales_ratio_rescale'] = test \
#    .groupby(['dept_id', 'store_id', 'd'])['sales_ratio'] \
#    .apply(lambda x: x / x.sum())

In [ ]:
forecast_df = top_forecast[['store_id', 'dept_id', 'd', 'yhat']]
forecast_df = pd.pivot_table(forecast_df, index=['store_id', 'dept_id'], columns='d', values='yhat')
forecast_df.columns = ['F' + str(int(c) + 1) for c in range(28)]
forecast_df = forecast_df.reset_index()

In [ ]:
sales_train_validation = pd.read_csv('../data/raw/sales_train_validation.csv')
sales_train_validation = sales_train_validation[['id', 'store_id', 'dept_id']]

In [ ]:
forecast_df = pd.merge(sales_train_validation, forecast_df)
forecast_df.drop(columns=['store_id', 'dept_id'], inplace=True)

In [ ]:
forecast_df.set_index('id', inplace=True)
top_down_ratio.set_index('id', inplace=True)
forecast_df = forecast_df * top_down_ratio
forecast_df.dropna(inplace=True)

In [ ]:
res = pd.read_csv('../data/raw/sample_submission.csv')
res.set_index('id', inplace=True)
res.update(forecast_df)
res.reset_index(inplace=True)

In [ ]:
---------

In [ ]:
forecast_df = pd.merge(top_down_ratio, top_forecast.drop(columns='id'))

In [ ]:
forecast = np.array(forecast_df['sales_ratio_softmax'] * forecast_df['yhat'])

In [ ]:
output_id = forecast_df.id.unique()
forecast_df = pd.DataFrame(forecast.reshape(output_id.shape[0], 28))
forecast_df.columns = ['F' + str(int(c) + 1) for c in forecast_df.columns]
forecast_df['id'] = output_id
if SUBMISSION_TYPE == 'validation':
    forecast_df['id'] = forecast_df['id'].str.replace('evaluation', 'validation')
forecast_df.set_index('id', inplace=True)

In [ ]:
res = pd.read_csv('../data/raw/sample_submission.csv')
res.set_index('id', inplace=True)
res.update(forecast_df)
res.reset_index(inplace=True)

In [ ]:
res.to_csv('../data/submission/Prophet_top_down_lgbm.csv', index=False)